# Lab 1 - Fault Injection Calibration

---
NOTE: This lab references some (commercial) training material on [ChipWhisperer.io](https://www.ChipWhisperer.io). You can freely execute and use the lab per the open-source license (including using it in your own courses if you distribute similarly), but you must maintain notice about this source location. Consider joining our training course to enjoy the full experience.

---

**SUMMARY:** *Now that we've seen how fault injection works, let's use it on some real hardware. In this lab, we'll be doing a sort of "calibration" by finding successful settings on simple firmware so that we can more easily do fault injection against more complicated firmware.*

**LEARNING OUTCOMES:**

* Understanding voltage glitch settings
* Exploring ChipWhisperer's glitch module
* Building a voltage glitch and crash map.

## \#HARDWARE

Connection is the same as before. There's no simulated version of the glitch labs, so you'll need hardware to run them.

In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CW308_SAM4S'
SS_VER = 'SS_VER_2_1'

In [ ]:
%run "../Setup_Scripts/Setup_Generic.ipynb"

We're using a different bit of firmware called `simpleserial-glitch`. This firmware contains different pieces of code that are designed to be easy to glitch, which is very useful when working with a target for the first time

In [ ]:
%%bash -s "$PLATFORM" "$SS_VER"
cd ../../hardware/victims/firmware/simpleserial-glitch
make PLATFORM=$1 CRYPTO_TARGET=NONE SS_VER=$2 -j

In [ ]:
fw_path = "../../hardware/victims/firmware/simpleserial-glitch/simpleserial-glitch-{}.hex".format(PLATFORM)
cw.program_target(scope, prog, fw_path)
if SS_VER=="SS_VER_2_1":
    target.reset_comms()

Glitching can cause a target to crash, which can also cause garbage on the serial lines. We'll define a quick function to reset the target and clear the serial buffer:

In [ ]:
def reboot_flush():
    reset_target(scope)
    target.flush()

There's a few different demos that this firmware can run. For this lab, the one we care about is triggered by the `"g"` command in simpleserial. The code for that looks like:

```C
uint8_t glitch_loop(uint8_t cmd, uint8_t scmd, uint8_t len, uint8_t* in)
{
    volatile uint16_t i, j;
    volatile uint32_t cnt;
    cnt = 0;
    trigger_high();
    for(i=0; i<50; i++){
        for(j=0; j<50; j++){
            cnt++;
        }
    }
    trigger_low();
    simpleserial_put('r', 4, (uint8_t*)&cnt);
    return (cnt != 2500) ? 0x10 : 0x00;
}
```

As you can see, there's not much going on here - it's just a loop that does 2500 increments of the `cnt` variable. Our goal will be to glitch this firmware so that `cnt != 2500`. This command takes no data, so you should send an empty array with the `"g"` command. It also returns 4 bytes (the `cnt` variable). Use what you learned in previous labs to send the `"g"` command and read `cnt` back. Make sure you remember to arm the scope and capture the trace.

In [ ]:
# ###################
# Add your code here to send the "g" command and read the response (Code Block 1)
# ###################
raise NotImplementedError("Add your code here, and delete this.")

You should get a bytearray containing `c4, 09, 00, 00` (`0x000009c4` as a 32-bit integer), which is in fact 2500. Now that we know what our goal is, let's take a closer look at glitching on the Husky.

The simpleserial targets also include a special `simpleserial_read_witherrors()` method that better handles glitches:

In [ ]:
# ###################
# Add your code here to send the "g" command (Code Block 2)
# ###################
raise NotImplementedError("Add your code here, and delete this.")

val = target.simpleserial_read_witherrors('r', 4, timeout=50, glitch_timeout=1)
print(val)

This method will tell you if you got a valid packet back, as well as the return value from the simpleserial function on the target.

## Voltage Glitch Hardware

Before we get to glitching, let's quickly run through the ChipWhisperer's glitching hardware. Here's a block diagram of the glitch module:

![glitch](img/cwlitepro_glitch.png)

Don't worry, we'll break this down into smaller chunks as we go along.

To start, the Husky's glitch module is quite power hungry so, by default, it's disabled. Let's fix that:

In [ ]:
scope.glitch.enabled = True

The goal with the Husky's glitch module is to precisely control a transistor so that we can short a target's Vcc pin to ground for small time periods. The way the Husky does that is by manipulating a clock using phase shifts. This means that we need a clock source for our glitch module. You can see the clock input in the top left of the block diagram:

![](img/glitch_inputs.png)

This clock can either come from the Husky's PLL (which also clocks the target/ADC), or from an external source. We're providing a clock for the target, so we can just use the PLL for this lab:

In [ ]:
scope.glitch.clk_src = "pll"

Next, this clock is sent through two phase shifts to control its properties

![A2_4](img/Glitchgen-phaseshift.png)

The first phase shift will shift the glitch stream in time. This phase shift can between -49.8% (49.8% backwards) and +49.8% (49.8% forwards) and can be controlled by `scope.glitch.offset`. The second phase shift controls the width of the glitch stream. This phase shift can also technically be between -50% and +50%, but negative values here are redundant. This phase shift is controlled by `scope.glitch.width`.

These values map a bit differently than earlier ChipWhisperers. For `scope.glitch.offset`, `[-49.8%, 49.8%]` maps to `[0, scope.glitch.phase_shift_steps]`. For `scope.glitch.width`, `[0, 49.8%]` maps to `[0, scope.glitch.phase_shift_steps//2]`.

`phase_shift_steps` depends on the input clock, as well as a Voltage Controlled Oscillator (VCO) internal to the Husky. This VCO can be controlled by `scope.clock.update_fpga_vco()`, and is adjustable between 600MHz and 1200MHz, with higher values giving finer steps on `width` and `offset`. We don't need super precise glitches, so we can leave this at its default.

The other important input for the glitch module is the trigger:

![trig](img/glitch_trigger.png)

This is a bit different from the trigger for the ADC. First of all, you can see that there's no rising/falling/high/low option. Instead, `trigger` is always rising edge and you can think of it as being high for a full glitch. We also have 4 different options for how to trigger the glitch:

* manual - trigger a glitch via `scope.glitch.manual_trigger()`
* ext_single - trigger a glitch on a rising edge when the scope is armed
* ext_continuous - trigger a glitch on a rising edge
* continuous - Continuously trigger a glitch

Most of the time, you'll want to use `ext_single`, as the trigger allows us to precisely place glitches and arming the scope gives us more control over when we're glitching. 

In [ ]:
scope.glitch.trigger_src = "ext_single"

Next, `ext_offset` allows us to delay the trigger by multiple clock cycles. This is very useful, as the operation you want to glitch is usually after the trigger, not right on top of it. Repeat is more situational. As the name suggests, it repeats the trigger multiple times. This can be combined with the `enable_only` output setting to glitch longer than one clock cycle.

We're almost there! The final part is the output stage:

![](img/glitch_output.png)

We're got 5 options here:

* clock_only - Feed the Input Clock directly into the output
* clock_xor - AND the Input Clock and Glitch Trigger, then XOR with the Input Clock
* clock_or - AND the Input Clock and Glitch Trigger, then OR with the Input Clock
* glitch_only - AND the Input Clock and Glitch Trigger
* enable_only - Glitch Trigger only

`clock_only`, `clock_xor`, and `clock_or` are mostly useful when clock glitching - trying to oscillate the Vcc pin like that is way too strong. `glitch_only` and `enable_only` can both be useful when glitching, but generally lend themselves to different strategies. If you're glitching without a target clock, you can crank up `scope.clock.clkgen_freq` and bypass the 50% restriction on the phase shift blocks. For this lab, however, we'll be using `glitch_only`, as we want to make use of the `width` and `offset` blocks and a 50% width is plenty strong enough to glitch this target.

In [ ]:
scope.glitch.output = "glitch_only"

All that's left now is to enable the glitch transistor that we'll be using. We generally recommend the high-power glitch transistor, as that seems to be the most reliable on the Husky:

In [ ]:
scope.io.glitch_hp = True

This setup is pretty consistent, so we've got a convenience method that recreates what we've done above: `scope.vglitch_setup('hp')`.

In [ ]:
scope.vglitch_setup?

We can also print the glitch module to quickly see our settings:

In [ ]:
print(scope.glitch)

Now that we know how the glitch module works, let's see if we can glitch our target. Basically, what we want to do here is iterate through `width`, `offset`, and `ext_offset` until we find settings that work. We'll also want to record whether or not each setting was successful, crashed the target, or did nothing. ChipWhisperer includes a `GlitchController` to make this easier:

In [ ]:
import chipwhisperer as cw
gc = cw.GlitchController(groups=["success", "reset", "normal"], parameters=["width", "offset", "ext_offset", "tries"])
gc.display_stats()

You can see we've added a `"tries"` parameter as well. This is just to make it easy to try each setting multiple times. We can also do a real-time glitch plot as well, so you can see which settings are working in real time:

In [ ]:
gc.glitch_plot(plotdots={"success":"+g", "reset":"xr", "normal":None}, x_index="width", y_index="offset")

Next, we need to set the ranges for these parameters. Usually the best strategy is to start with a large range and work our way down as we learn more about the target:

In [ ]:
gc.set_range("tries", 1, 1) # one try for each setting
gc.set_range("ext_offset", 0, 30)

# Fill in your width/offset:
# ###################
# Add your code here to set width/offset/ext_offset from glitch_setting (Code Block 3)
# ###################
raise NotImplementedError("Add your code here, and delete this.")

scope.glitch.repeat = 1

# Stops Husky's error LEDs from flashing due to weird power traces
scope.adc.lo_gain_errors_disabled = True
scope.adc.clip_errors_disabled = True

We can also set how large we want our step size to be when iterating through our range. `scope.glitch.width` and `scope.glitch.offset` are likely several thousand, so we want a fairly large step size starting out. Something like 250 should work. Like with the parameter ranges, we can reduce this if required later on.

The glitch controller also allows you to set step sizes for individual parameters as well. This is very useful on Husky, as we probably want to increase `ext_offset`'s step size by 1 instead of 250.

In [ ]:
gc.set_global_step([10]) # reduce to fine tune glitching
gc.set_step("ext_offset", 1)
gc.set_step("tries", 1)

If you want to adjust width and offset, you can do so:

In [ ]:
#gc.set_range("width", 0, scope.glitch.phase_shift_steps // 2) # max range
#gc.set_range("offset", 0, scope.glitch.phase_shift_steps) # max range
#gc.set_step("width", 1)
#gc.set_step("offset", 1)

We can then get our parameters back with `gc.glitch_values()`. This is an iterator, so you can loop through it with `for`:

In [ ]:
i = 0
for glitch_setting in gc.glitch_values():
    print(glitch_setting)
    i += 1
    if i > 100:
        break

You should see a bunch of values printed out iterating through each of the parameters.

And that's pretty much it! All that's left is to make a loop to glitch our target. Things we need to do are:

1. Iterate through the loop
1. Assign our glitch_settings
1. Arm the scope
1. Send the `"g"` command
1. Call `scope.capture()`
1. Read back the result
1. Determine if we had a successful glitch, a crash, or if nothing happened. 

You can see below, that that's mostly what's happening. There's some extra stuff there to try to detect if the target is crashed before attempting a glitch, as this speeds up glitching a lot. You can also add `"success"` `"reset"` and `"normal"` with `gc.add()`, which will allow the glitch controller to keep track of the results and plot the glitch settings.

You'll be responsible for the bolded steps from above. Remember that the `2500` read back is actually a bytearray of the bytes that make up that number (`bytearray([c4, 09, 00, 00])`).

In [ ]:
import struct

#disable logging
cw.set_all_log_levels(cw.logging.CRITICAL)
scope.adc.timeout = 0.5 # prevent crashes from taking too long

reboot_flush()

# 1. Iterate through the loop
for glitch_setting in gc.glitch_values():
    
    # 2. Assign our glitch settings
    # ###################
    # Add your code here to set width/offset/ext_offset from glitch_setting (Code Block 4)
    # ###################
    raise NotImplementedError("Add your code here, and delete this.")

    target.flush() # flush garbage from serial lines
    
    # Try detecting if the target is crashed
    if scope.adc.state:
        gc.add("reset")
        reboot_flush()

    # 3. Arm the scope
    scope.arm()

    # 4. Send the "g" command
    target.simpleserial_write("g", bytearray([]))

    # Call scope.capture
    ret = scope.capture()
    
    # If trigger didn't fire, target is probably crashed
    if ret:
        gc.add("reset")
        reboot_flush()
    else:
        # 6. Read back the calculated number
        val = target.simpleserial_read_witherrors('r', 4, glitch_timeout=1, timeout=50)
        
        # If target sent us garabge, it probably crashed
        if val['valid'] is False:
            gc.add("reset")
            reboot_flush()
        else:
            # 7. Determine if we had a successful glitch
            # ###################
            # Add your code here to set width/offset/ext_offset from glitch_setting (Code Block 4)
            # ###################
            raise NotImplementedError("Add your code here, and delete this.")
            
print("Done glitching")

# reenable logging
cw.set_all_log_levels(cw.logging.WARNING)

Don't worry if you didn't get any successful glitches - you've still got some valuable information! You should see a big region where your width is too small to have any effect. Similarly, you probably have a region where you're only getting resets. Go back and narrow down your width range and step size, then rerun the glitch loop.

You may also want to increase range of the "tries" parameter. Glitching typically works a percentage of the time, so trying each setting multiple times can give you information about how often they work/cause resets.

The glitch controller includes a `calc` function for seeing how well your glitches worked. For example, the following will sort by the success rate while ignoring tries and ext_offset, allowing you to see which width/offset settings worked best. **You should record the locations you got successful glitches at. This will be helpful in later labs.**

In [ ]:
results = gc.calc(ignore_params=["tries"], sort="success_rate")
results

If you run through the lab again, you might want to try seeing which `ext_offset` worked so that you can also reduce the range on this.

In [ ]:
results = gc.calc(ignore_params=["tries", "offset", "width"], sort="success_rate")
results

In [ ]:
scope.dis()
target.dis()

In [ ]:
assert total_successes >= 1